In [516]:
import pandas as pd
import numpy as np
from haversine import haversine
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

In [517]:
df = pd.read_csv('C:/Users/user/github/Data/capstone/total_eda_real.csv')

In [518]:
df.지점명.unique()

array(['속초', '북춘천', '철원', '동두천', '파주', '대관령', '춘천', '백령도', '북강릉', '강릉',
       '동해', '서울', '인천', '원주', '울릉도', '수원', '영월', '충주', '서산', '울진', '청주',
       '대전', '추풍령', '안동', '상주', '포항', '군산', '대구', '전주', '울산', '창원', '광주',
       '부산', '통영', '목포', '여수', '흑산도', '완도', '고창', '순천', '홍성', '제주', '고산',
       '성산', '서귀포', '진주', '강화', '양평', '이천', '인제', '홍천', '태백', '정선군', '제천',
       '보은', '천안', '보령', '부여', '금산', '세종', '부안', '임실', '정읍', '남원', '장수',
       '고창군', '영광군', '김해시', '순창군', '북창원', '양산시', '보성군', '강진군', '장흥', '해남',
       '고흥', '의령군', '함양군', '광양시', '진도군', '봉화', '영주', '문경', '청송군', '영덕',
       '의성', '구미', '영천', '경주시', '거창', '합천', '밀양', '산청', '거제', '남해'],
      dtype=object)

In [519]:
len(df.지점명.unique())

95

In [520]:
df

,지점,위도,경도,지점명,일시,기온,풍속,풍향,습도,강수량
0,90,38.2509,128.5647,속초,2021-01-01 00:00,-6.3,2.6,290.0,30.0,0.0
1,90,38.2509,128.5647,속초,2021-01-01 01:00,-6.7,4.3,320.0,30.0,0.0
2,90,38.2509,128.5647,속초,2021-01-01 02:00,-6.7,2.0,340.0,26.0,0.0
3,90,38.2509,128.5647,속초,2021-01-01 03:00,-7.2,1.3,320.0,25.0,0.0
4,90,38.2509,128.5647,속초,2021-01-01 04:00,-7.6,0.8,180.0,25.0,0.0
...,...,...,...,...,...,...,...,...,...,...
1662115,295,34.8166,127.9264,남해,2022-12-30 20:00,3.5,1.7,290.0,52.0,0.0
1662116,295,34.8166,127.9264,남해,2022-12-30 21:00,2.3,1.4,270.0,60.0,0.0
1662117,295,34.8166,127.9264,남해,2022-12-30 22:00,0.8,1.1,270.0,67.0,0.0
1662118,295,34.8166,127.9264,남해,2022-12-30 23:00,0.8,0.7,290.0,65.0,0.0


In [521]:
df2 = pd.read_csv('C:/Users/user/Downloads/20230320154604/청운효자동_1시간기온_20220101_20221231.csv', index_col=False)

In [522]:
df2

,day,hour,forecast,value location:60_127
0,Start : 20220101,NaN,NaN,NaN
1,1,200.0,6.0,0.0
2,1,200.0,7.0,-2.0
3,1,200.0,8.0,-2.0
4,1,200.0,9.0,-2.0
...,...,...,...,...
739064,31,2300.0,61.0,-5.0
739065,31,2300.0,62.0,-5.0
739066,31,2300.0,63.0,-5.0
739067,31,2300.0,64.0,-5.0


In [523]:
d = df2[df2['day'].str.contains('Start')]

In [524]:
for i in d.index.tolist():
    d.day[i] = d.day[i].strip()[8:]

In [525]:
list = d.index.tolist()

In [526]:
len(list)

1449

In [527]:
for i in range(len(list)):
    if i == len(list)-1:
        df2.day[(df2.index >= list[i])] = d.day[738559]
    else:
        df2.day[(df2.index >= list[i]) & (df2.index < list[i+1])] = d[(d.index >= list[i]) & (d.index < list[i+1])].day.values[0]

In [528]:
df2 = df2.dropna()

In [529]:
df2.columns = ['day','hour','forecast','value']

In [530]:
df2.hour = df2.hour.astype('str')

In [531]:
df2 = df2.drop(df2[df2['value'] == 'value  location:60_127 Start : 20220101 '].index)

In [532]:
df2.value = df2.value.astype('float')

In [533]:
df2 = df2.drop(df2[df2['value'] <= -900].index)

In [534]:
df2.hour = df2.hour.astype('float')

In [536]:
df2.hour = df2.hour/100

In [540]:
df2.hour = df2.hour.astype('int')

In [542]:
df2

,day,hour,forecast,value
1,20220101,2,6.0,0.0
2,20220101,2,7.0,-2.0
3,20220101,2,8.0,-2.0
4,20220101,2,9.0,-2.0
5,20220101,2,10.0,-2.0
...,...,...,...,...
739063,20221231,23,60.0,-4.0
739064,20221231,23,61.0,-5.0
739065,20221231,23,62.0,-5.0
739066,20221231,23,63.0,-5.0


In [548]:
df2.day = pd.to_datetime(df2.day)

In [549]:
df2

,day,hour,forecast,value
1,2022-01-01,2,6.0,0.0
2,2022-01-01,2,7.0,-2.0
3,2022-01-01,2,8.0,-2.0
4,2022-01-01,2,9.0,-2.0
5,2022-01-01,2,10.0,-2.0
...,...,...,...,...
739063,2022-12-31,23,60.0,-4.0
739064,2022-12-31,23,61.0,-5.0
739065,2022-12-31,23,62.0,-5.0
739066,2022-12-31,23,63.0,-5.0
